Token Counter

In [1]:
from transformers import AutoTokenizer

# tokens are a model's context window / memory.  The model can only "see" this many tokens at a time.
# different models have different context window sizes.
# smaller models tend to have smaller context windows.
# larger models tend to have larger context windows.
# context window sizes can range from 512 tokens to 32768 tokens or more.
# for reference, 1 token is approximately 4 characters of English text, so 1000 tokens is approximately 750 words.
checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant specialized in converting a unit to another unit.",
    },
    {"role": "user", "content": "What is the conversion of 10 miles to kilometers?"},
]
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
tokens = tokenizer(input_text).input_ids
print(f"Token count: {len(tokens)}")

/home/edo/miniconda3/envs/deeplearn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token count: 36


In [2]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("HuggingFaceTB/SmolLM2-360M-Instruct")

# HuggingFaceTB/SmolLM2-360M-Instruct has a context window of 2048 tokens
print(f"HuggingFaceTB/SmolLM2-360M-Instruct tokenizer.model_max_length: {tokenizer.model_max_length}")
print(f"HuggingFaceTB/SmolLM2-360M-Instruct config.max_position_embeddings: {config.max_position_embeddings}")

HuggingFaceTB/SmolLM2-360M-Instruct tokenizer.model_max_length: 8192
HuggingFaceTB/SmolLM2-360M-Instruct config.max_position_embeddings: 8192


Question and Answer Sampling from Model

https://huggingface.co/HuggingFaceTB/SmolLM2-360M-Instruct

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def model_generate(messages, num_return_sequences: int | None = None, temperature: float = 0, max_new_tokens=50, repetition_penalty=None):
    checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"

    device = "cuda"  # for GPU usage or "cpu" for CPU usage
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    # for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
    model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

    input_text = tokenizer.apply_chat_template(messages, tokenize=False)
    tokens = tokenizer(input_text).input_ids
    print(f"Token count: {len(tokens)}")
    #print(input_text)
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)

    do_sample = True if temperature > 0 else False
    n_return_sequences = num_return_sequences if num_return_sequences is not None else 1

    outputs = model.generate(
        **inputs, 
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature if do_sample else 1.0,
        num_return_sequences=n_return_sequences, 
        repetition_penalty=repetition_penalty if do_sample else None,
        eos_token_id=tokenizer.eos_token_id
    )

    #print(outputs)

    #print(tokenizer.decode(outputs[0], skip_special_tokens=True))

    # decoded_outputs is a list of strings corresponding to each generated output
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    if n_return_sequences == 1:
        # only one output, print it directly from the list of strings
        print(decoded_outputs[0])
    else:
        for i, output in enumerate(decoded_outputs):
            print(f"--- **Result {i+1}** --------------------")
            print(output)
        print("-----------------------------------")

In [4]:
system_prompt = (
    "You are an assistant specialized in unit conversions and will be answering a unit conversion question or request. Follow these rules when responding:\n"
    "- Be concise and wrap the numerical result in <answer> tags. Example: <answer>5.86</answer>\n"
    "- Double check your calculations\n"
)

# hyperparameters
temp = 0.6
repetition_penalty = 1.1  # override repetition penalty to reduce repetitive outputs
# Set a reasonable limit for the number of tokens to generate. Prevents infinite generation.
max_new_tokens = 150
num_return_sequences = 10 # number of samples to generate

question = "How do we translate 7 KB into bit?"

In [5]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question},
]

model_generate(
    messages,
    temperature=temp,
    repetition_penalty=repetition_penalty,
    max_new_tokens=max_new_tokens,
    num_return_sequences=num_return_sequences,
)

Token count: 79


TypeError: transformers.generation.utils.GenerationMixin.generate() argument after ** must be a mapping, not Tensor

Use few shot examples:

Example 1: Convert 10 kilometers to miles. 1 kilometer = 0.621371 miles. Answer: 6.21371 miles.

Example 2: Convert 2 liters to gallons. 1 liter = 0.264172 gallons. Answer: 0.528344 gallons.

Now, convert 5 meters to feet. 1 meter = 3.28084 feet. Answer:


In [120]:
messages2 = [
    {"role": "system", "content": system_prompt},
    {
        "role": "user",
        "content": "Can you change 9 metric ton to its equivalent in kg?",
    },
    {
        "role": "assistant",
        "content": (
            "Conversion calculation: 9 metric ton * 1000 kg/metric ton = 9000 kg\n"
            "<answer>9000.0</answer>"
        ),
    },
    {"role": "user", "content": "What is the conversion of 4 mph to ft/s?"},
    {
        "role": "assistant",
        "content": (
            "Conversion calculation: 4 mph * 5280 ft/mile * (1 hour / 3600 seconds) = 5.867 ft/s\n"
            "<answer>5.867</answer>"
        ),
    },
    {"role": "user", "content": "Convert the measurement of 5 kB into bit."},
    {
        "role": "assistant",
        "content": (
            "Conversion calculation: 5 kB * 1024 bytes/kB * 8 bits/byte = 40960 bits\n"
            "<answer>40960.0</answer>"
        ),
    },
    {
        "role": "user",
        "content": "Can you provide the conversion value from years to week for 7 units?",
    },
    {
        "role": "assistant",
        "content": (
            "Conversion calculation: 7 years * 365 days/year * 1 week/7 days = 364.24219878125 weeks\n"
            "<answer>364.24219878125</answer>"
        ),
    },
    {"role": "user", "content": "Please convert 1 ft into cm."},
    {
        "role": "assistant",
        "content": (
            "Conversion calculation: 1 ft * 12 in/ft *  2.54 cm/in = 30.48 cm\n"
            "<answer>30.48</answer>"
        ),
    },
    {"role": "user", "content": "What is the equivalent of 9 milliliter in mm^3?"},
    {
        "role": "assistant",
        "content": (
            "Conversion calculation: 9 milliliter * 1000 mm^3/milliliter = 9000 mm^3\n"
            "<answer>9000</answer>"
        ),
    },
    # the actual question
    {"role": "user", "content": question},
]

model_generate(
    messages2,
    temperature=temp,
    repetition_penalty=repetition_penalty,
    max_new_tokens=max_new_tokens,
)

Token count: 501
system
You are an assistant specialized in unit conversions and will be answering a unit conversion question or request. Follow these rules when responding:
- Be concise and wrap the numerical result in <answer> tags. Example: <answer>5.86</answer>
- Double check your calculations

user
Can you change 9 metric ton to its equivalent in kg?
assistant
Conversion calculation: 9 metric ton * 1000 kg/metric ton = 9000 kg
<answer>9000.0</answer>
user
What is the conversion of 4 mph to ft/s?
assistant
Conversion calculation: 4 mph * 5280 ft/mile * (1 hour / 3600 seconds) = 5.867 ft/s
<answer>5.867</answer>
user
Convert the measurement of 5 kB into bit.
assistant
Conversion calculation: 5 kB * 1024 bytes/kB * 8 bits/byte = 40960 bits
<answer>40960.0</answer>
user
Can you provide the conversion value from years to week for 7 units?
assistant
Conversion calculation: 7 years * 365 days/year * 1 week/7 days = 364.24219878125 weeks
<answer>364.24219878125</answer>
user
Please conver

# Part 4
Function to test implementations

In [1]:
from homework.datagen import *
from homework.cot import *
from homework.data import *

trainset = Dataset("train") # about a 1000 train samples
trainset = trainset[:64] # get 64 samples for quick testing

oversample = 10
temperature = 0.6

model = CoTModel()

trainset_size = range(len(trainset))
prompts = [model.format_prompt(trainset[i][0]) for i in trainset_size]

generations = model.batched_generate(prompts, num_return_sequences=oversample, temperature=temperature)

if isinstance(generations[0], list):
    # multiple generations per prompt
    print("=== Generations ===")
    for i, gen in enumerate(generations):
        for j, g in enumerate(gen):
            print(f"Prompt {i + 1}, Sample {j + 1}: \n {g}")
elif isinstance(generations[0], str):
    # single generation per prompt
    for i, g in enumerate(generations):
        print(f"Prompt {i + 1}: \n {g}")
else:
    # unexpected format
    print("Unexpected generation format.")

/home/edo/anaconda3/envs/deeplearn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BaseLLM __initi__: Loaded model on device: cuda using model (HuggingFaceTB/SmolLM2-360M-Instruct) with float32


LLM Running on Micro Batches 32: 100%|██████████| 2/2 [01:19<00:00, 39.92s/it]

=== Generations ===
Prompt 1, Sample 1: 
 Conversion used: 60 minutes/hour
Calculation: 2 * 60 = 120 minutes
<answer>120</answer>
Prompt 1, Sample 2: 
 Calculation: 2 * 60 = 120
<answer>120</answer>
Prompt 1, Sample 3: 
 Conversion used: 60 minutes/hour
Calculation: 2 * 60 = 120
<answer>120</answer>
Prompt 1, Sample 4: 
 Conversion used: 60 min/hour
Calculation: 2 * 60 = 120
<answer>120</answer>
Prompt 1, Sample 5: 
 Conversion used: 60 minutes/hour
Calculation: 2 * 60 = 120
<answer>120</answer>
Prompt 1, Sample 6: 
 Conversion used: 60 minutes/hour
Calculation: 2 * 60 = 120
<answer>120</answer>
Prompt 1, Sample 7: 
 Conversion used: 60 minutes/hour
Calculation: 2 * 60 = 120
<answer>120</answer>
Prompt 1, Sample 8: 
 Conversion used: 60 minutes/hour
Calculation: 2 * 60 = 120
<answer>120</answer>
Prompt 1, Sample 9: 
 Conversion used: 60 minutes/hour
Calculation: 2 * 60 = 120
<answer>120</answer>
Prompt 1, Sample 10: 
 Conversion used: 60 minutes/hour
Calculation: 2 * 60 = 120 minutes
<

Test rft data model

In [5]:
from homework.data import Dataset
rft_trainset = Dataset("rft")

print(f"RFT trainset size: {len(rft_trainset)}")
print("First RFT trainset example:")
# print rft trainset
for record in rft_trainset:
    print(record)

RFT trainset size: 893
First RFT trainset example:
['Can you change 2 hour to its equivalent in min?', 120.0, 'Conversion used: 60 minutes/hour\nCalculation: 2 * 60 = 120\n<answer>120</answer>']
['What is the conversion of 2 hour to seconds?', 7200.0, 'Conversion used: 3600 seconds/hour\nCalculation: 2 * 3600 = 7200\n<answer>7200</answer>']
['How many gram are there per 6 kg?', 6000.0, 'Conversion used: 1000 g/kg\nCalculation: 6 * 1000 = 6000\n<answer>6000</answer>']
['What is 4 months in hour?', 2921.9375902499996, 'Conversion used: 1 month ≈ 30.5 days\n1 day ≈ 24 hours\nCalculation: 4 * 30.5 * 24 ≈ 3024\n<answer>3024</answer>']
['How much is 10 pint when converted to milliliter?', 4731.764730000003, 'Conversion used: 473.176 ml/pint\nCalculation: 10 * 473.176 = 4731.76\n<answer>4731.76</answer>']
['Convert the measurement of 6 l into cm^3.', 5999.999999999999, 'Conversion used: 1000 cc/liter\nCalculation: 6 * 1000 = 6000\n<answer>6000</answer>']
['Can you change 9 metric ton to its e